In [1]:
from time import time

import numpy as np
import pandas as pd
from da4ml.cmvm.api import solve

In [2]:
# Copying data from hcmvm paper

data = [[2, 8.8, 3.5, 9.7, 8.7, 3.6, 8.2, 4.4, 8.2, 3.7, 9.0, 3.1, 8.8, 3.1],
[4, 32.1, 5.9, 31.2, 31.7, 5.8, 27.6, 7.8, 28.1, 5.7, 32.8, 4.1, 32.1, 4.1],
[6, 68.0, 7.6, 66.1, 66.5, 7.7, 57.3, 10.0, 58.2, 7.0, 68.1, 5.0, 66.8, 5.0],
[8, 116.4, 9.2, 113.2, 114.1, 9.2, 96.3, 11.9, 99.5, 7.1, 119.7, 5.1, 117.2, 5.1],
[10, 175.7, 10.7, 172.4, 172.0, 10.5, 143.5, 13.2, 146.9, 8.0, 175.8, 6.0, 157.7, 6.0],
[12, 246.5, 12.0, 241.6, 240.9, 11.7, 200.4, 14.6, 206.8, 8.0, 247.1, 6.0, 241.6, 6.0],
[14, 327.1, 13.2, 322.9, 320.0, 13.0, 264.3, 15.5, 274.8, 8.0, 330.2, 6.0, 324.0, 6.0],
[16, 417.9, 14.4, 412.4, 407.5, 14.0, 338.3, 16.3, 353.3, 8.0, 431.0, 6.0, 423.2, 6.0]]

col0 = ['[12]','[5]', 'H2MC', 'hcmvm', 'hcmvm-dc2', '[27]-DC0', 'hcmvm-dc']
col1 = ['adder', 'step']
col = pd.MultiIndex.from_product([col0, col1], names=['method', 'type'])
col = [('','N')] + list(col)
del col[4]
col = pd.MultiIndex.from_tuples(col)

df_hcmvm = pd.DataFrame(data, columns=col)
df_hcmvm.set_index(('','N'), inplace=True)
df_hcmvm.index.name = 'N'

# Runtime in hcmvm is only in a plot, extracting data points from there

_hcmvm_times = np.array([0,532,800,1009,1185,1342,1485,1612])
_hcmvm_dc_times = np.array([0,530,819,1026,1242,1342,1466,1584])

def convert(pixel):
    return 1e5**(pixel/1586) * 10
hcmvm_times = convert(_hcmvm_times)
hcmvm_dc_times = convert(_hcmvm_dc_times)
df_hcmvm[('hcmvm', 'cpu')] = hcmvm_times
df_hcmvm[('hcmvm-dc', 'cpu')] = hcmvm_dc_times
df_hcmvm.sort_index(inplace=True, axis=1)

In [3]:
df_hcmvm.drop(columns=['[5]','H2MC','[12]','[27]-DC0'], inplace=True)

In [4]:
# One run to compile the solver, just in case the cache does not exist

sol = solve(np.random.randint(-128,128,(8,8)).astype(np.float32), search_all_decompose_dc=False)

In [5]:
_hcmvm_times = np.array([0,532,800,1009,1085,1342,1485,1612])
_hcmvm_dc_times = np.array([0,530,819,1026,1242,1342,1466,1584])
def convert(pixel):
    return 1e4**(pixel/1557) * 10
hcmvm_times = convert(_hcmvm_times)
hcmvm_dc_times = convert(_hcmvm_dc_times)

In [11]:
N_trail = 1000

r = {}
for dc in [-1, 2, 0]: # delay constraint
    _dc_str = f'-dc{dc}' if dc != -1 else ''
    for N in df_hcmvm.index:
        _steps = []
        _adders = []
        _time = []
        print(f'Running N={N}, dc={dc}')
        for _ in range(N_trail):
            arr = np.floor(np.random.rand(N, N).astype(np.float32) * 127) + 129
            t0 = time()
            sol = solve(arr, method0='wmc', hard_dc=dc, search_all_decompose_dc=False)
            t = time() - t0
            _steps.append(sol.latency[1])
            _adders.append(sol.cost)
            _time.append(t*1000)
        r.setdefault(N,{}).update({
            (f'da4ml{_dc_str}','adder'): np.mean(_adders),
            (f'da4ml{_dc_str}','step'): np.mean(_steps),
            (f'da4ml{_dc_str}','time'): np.mean(_time),
        })

Running N=2, dc=-1
Running N=4, dc=-1
Running N=6, dc=-1
Running N=8, dc=-1
Running N=10, dc=-1
Running N=12, dc=-1
Running N=14, dc=-1
Running N=16, dc=-1
Running N=2, dc=2
Running N=4, dc=2
Running N=6, dc=2
Running N=8, dc=2
Running N=10, dc=2
Running N=12, dc=2
Running N=14, dc=2
Running N=16, dc=2
Running N=2, dc=0
Running N=4, dc=0
Running N=6, dc=0
Running N=8, dc=0
Running N=10, dc=0
Running N=12, dc=0
Running N=14, dc=0
Running N=16, dc=0


In [12]:
df_da4ml = pd.DataFrame.from_dict(r, orient='index')
df_da4ml.index.name = 'N'

In [13]:
col0 = ['hcmvm']
col1 = ['-', '0', '2']
col2 = ['adder', 'cpu time [ms]', 'step']
cols_hcmvm = pd.MultiIndex.from_product([col0, col1, col2], names=['method', 'dc', 'type'])
cols_hcmvm = cols_hcmvm.drop(('hcmvm', '2', 'cpu time [ms]'))

col0 = ['da4ml']
col1 = ['-', '2', '0']
col2 = ['adder', 'step', 'cpu time [ms]']
cols_da4ml = pd.MultiIndex.from_product([col0, col1, col2], names=['method', 'dc', 'type'])

cols = cols_hcmvm.append(cols_da4ml)

In [14]:
df = pd.concat([df_hcmvm, df_da4ml], axis=1).round(1)
df.columns = cols
df.sort_index(inplace=True, axis=1)

In [15]:
df[sorted(df.columns, key=lambda x: (x[1], x[0], len(x[2])))]  # CPU time for hcmvm, DC=2 is missing

method da4ml                      hcmvm                      da4ml         \
dc         -                          -                          0          
type    step  adder cpu time [ms]  step  adder cpu time [ms]  step  adder   
N                                                                           
2        3.3    8.6           0.1   4.4    8.2          10.0   3.1    9.1   
4        6.0   29.2           0.3   7.8   27.6         475.5   4.1   34.3   
6        8.3   59.0           0.7  10.0   57.3        3327.1   5.0   73.3   
8        9.3   98.1           1.5  11.9   96.3       15169.1   5.1  124.9   
10      10.8  146.5           2.9  13.2  143.5       54426.9   6.0  188.5   
12      11.7  203.5           5.0  14.6  200.4      170125.4   6.0  263.5   
14      12.2  269.3           8.5  15.5  264.3      480384.4   6.0  349.7   
16      12.9  343.2          13.6  16.3  338.3     1207722.7   6.0  446.9   

method               hcmvm                      da4ml                       \
dc                       0                          2                        
type   cpu time [ms]  step  adder cpu time [ms]  step  adder cpu time [ms]   
N                                                                            
2                0.1   3.1    8.8          10.0   3.3    8.7           0.1   
4                0.4   4.1   32.1         468.7   5.9   29.6           0.3   
6                1.0   5.0   66.8        3819.2   6.7   62.4           0.7   
8                2.4   5.1  117.2       17161.4   7.0  101.8           1.5   
10               4.8   6.0  157.7       82321.0   7.8  152.2           2.9   
12               8.9   6.0  241.6      170125.4   8.0  214.6           5.4   
14              15.0   6.0  324.0      418494.5   8.0  278.6           9.3   
16              24.2   6.0  423.2      985586.7   8.0  357.3          15.4   

method hcmvm         
dc         2         
type    step  adder  
N                    
2        3.7    8.2  
4        5.7   28.1  
6        7.0   58.2  
8        7.1   99.5  
10       8.0  146.9  
12       8.0  206.8  
14       8.0  274.8  
16       8.0  353.3